In [1]:
%env API_KEY = cc94cceddad84259873b40731c6a5fea
%env USER = user1
%env PASSWD = #password99

env: API_KEY=cc94cceddad84259873b40731c6a5fea
env: USER=user1
env: PASSWD=#password99


## Importing the libraries

In [3]:
import pandas as pd

import time

import os

API_KEY =  os.environ['API_KEY']

USER = os.environ['USER']

PASSWD = os.environ['PASSWD']

ModuleNotFoundError: No module named 'pandas'

### newsapi is the client libraries of NewsAPI

In [3]:
# Importing the API client of newsapi 

from newsapi import NewsApiClient

In [4]:
# function to get the data from API client

def get_data():
    try:
        newsapi = NewsApiClient(api_key = API_KEY)
        
        source = newsapi.get_sources(language = 'en')
        
        # wait for 1 second after making the GET request
        
        time.sleep(1)
        
        df = pd.DataFrame(columns=['id', 'name', 'description', 'url', 'category', 'countary'])
        
        for src in source['sources']:
            
            id_ = src['id']
            
            name = src['name']
            
            description = src['description']
            
            url = src['url']
            
            category = src['category']
            
            country = src['country']
            
            df = df.append({
                'id' : id_,
                'name' : name,
                'description' : description,
                'url' : url,
                'category' : category,
                'countary' : country
            },
                ignore_index=True)
            
        return df
    
    except Error as e:
        
        print(e)


In [5]:
# storing the dataframe in df

df = get_data()

In [6]:
# checking the head of the dataframe

df.head()

,id,name,description,url,category,countary
0,abc-news,ABC News,"Your trusted source for breaking news, analysi...",https://abcnews.go.com,general,us
1,abc-news-au,ABC News (AU),"Australia's most trusted source of local, nati...",http://www.abc.net.au/news,general,au
2,al-jazeera-english,Al Jazeera English,"News, analysis from the Middle East and worldw...",http://www.aljazeera.com,general,us
3,ars-technica,Ars Technica,The PC enthusiast's resource. Power users and ...,http://arstechnica.com,technology,us
4,associated-press,Associated Press,The AP delivers in-depth coverage on the inter...,https://apnews.com/,general,us


In [7]:
# Checking the target values(category) of the data

df['category'].unique()

array(['general', 'technology', 'business', 'sports', 'entertainment',
       'health', 'science'], dtype=object)

In [8]:
# counting the total values of every category

df['category'].value_counts()

general          41
sports           11
technology       10
entertainment     8
business          7
science           3
health            1
Name: category, dtype: int64

In [9]:
# droping the non relevent columns from dataframe

df_clean = df.drop(['id', 'name', 'url', 'countary'], axis = 1)

In [10]:
# checking the total number of rows in df

df_clean.count()

description    81
category       81
dtype: int64

# Text Preprocessing

## Importing the nltk libraries for text preprocessing

In [11]:
from nltk.corpus import stopwords

import string

import re

In [12]:
# function to remove the stopwords, punctuation and creating the word tokens

def clean_txt(text):
    
#     text = re.sub('[0-9]', '', text)
    
    text = "".join([char.lower() for char in text if char not in string.punctuation])
    
    tokens = re.split('\W+', text)
    
    text = [word for word in tokens if word not in stopwords.words('english')]
    
    return text

In [13]:
# applying the clean_txt function on the description column

df_clean['description_clean'] = df_clean['description'].apply(lambda x : clean_txt(x))

In [14]:
# checking the head

df_clean.head()

,description,category,description_clean
0,"Your trusted source for breaking news, analysi...",general,"[trusted, source, breaking, news, analysis, ex..."
1,"Australia's most trusted source of local, nati...",general,"[australias, trusted, source, local, national,..."
2,"News, analysis from the Middle East and worldw...",general,"[news, analysis, middle, east, worldwide, mult..."
3,The PC enthusiast's resource. Power users and ...,technology,"[pc, enthusiasts, resource, power, users, tool..."
4,The AP delivers in-depth coverage on the inter...,general,"[ap, delivers, indepth, coverage, internationa..."


# Inserting the data into MySQL Database

In [15]:
# importing the connect and error methods from mysql.connector libraries to interect with the relational database

from mysql.connector import connect, errors

In [16]:
# function to create the database connection

def db_connection(host, usr, passwd):
    
    try:
        
        conn = connect(host = host, user = usr, passwd = passwd, buffered = True, database='news_data')
        
    except Error as e:
        
        print(e)
        
    else:
        
        print('Connection is Successful!')
        
    return conn
    

In [32]:
# function to create table in database

def create_table(db_name, table_name, columns):
    
    try:
        
        cursor = conn.cursor()
    
        create_table = f'CREATE TABLE IF NOT EXISTS {table_name}({columns}) ;'

        cursor.execute(create_table)
    
        print(f"'{table_name}' Table created successfully!")
    
        cursor.close()
    
    except Error as e:
        
        print(e)
    

In [18]:
# function to insert the new row into table

def insert_query(table_name, columns, column_values, db_name):
    
    try:
        cursor = conn.cursor()
    
        insert_query = f'INSERT INTO {table_name} ({columns}) VALUES({column_values});'
        
        print(f'Data inserted successfully in {table_name}!')

        cursor.execute(insert_query)
        
        cursor.close()
            
    except Error as e:
        
        print(e)

In [19]:
# function to insert the data into table

def insert_data():
    
    cursor = conn.cursor()

    for index, rows in df_clean.iterrows():

        values = f"'{rows.category}', ('{','.join(rows.description_clean)}')" 

        cursor.execute('SELECT description FROM news_data_clean')

        description_data = cursor.fetchall()

        temp = [row[-1] for row in description_data]

        if ','.join(rows.description_clean) not in temp:

            cursor.execute(insert_query(table_name = 'news_data_clean', columns = 'category, description', column_values =values, db_name = 'news_data'))

In [20]:
# function to show the content of table

def show_tables_data(table_name):
    
    cursor = conn.cursor()
    
    cursor.execute(f'SELECT * FROM {table_name};')
    
    print(f"Total numbers of rows in table {cursor.rowcount}")
    
    results = cursor.fetchall()

    for row in results:
            print("category = ", row[0], )
            print("description = ", row[1])
            print("\n")

In [21]:
conn = db_connection('localhost', USER, PASSWD)

Connection is Successful!


In [31]:
create_table(db_name = 'news_data', table_name = 'news_data_clean', columns = 'category VARCHAR(20), description VARCHAR(500)')

NameError: name 'Error' is not defined

In [25]:
insert_data()

Data inserted successfully in news_data_clean!
Data inserted successfully in news_data_clean!
Data inserted successfully in news_data_clean!
Data inserted successfully in news_data_clean!
Data inserted successfully in news_data_clean!
Data inserted successfully in news_data_clean!
Data inserted successfully in news_data_clean!


In [26]:
# to affect the changes in database

conn.commit()

In [27]:
show_tables_data('news_data_clean')

Total numbers of rows in table 87
category =  general
description =  trusted,source,breaking,news,analysis,exclusive,interviews,headlines,videos,abcnewscom


category =  general
description =  australias,trusted,source,local,national,world,news,comprehensive,independent,indepth,analysis,latest,business,sport,weather


category =  general
description =  news,analysis,middle,east,worldwide,multimedia,interactives,opinions,documentaries,podcasts,long,reads,broadcast,schedule


category =  technology
description =  pc,enthusiasts,resource,power,users,tools,love,without,computing,religion


category =  general
description =  ap,delivers,indepth,coverage,international,politics,lifestyle,business,entertainment,news


category =  business
description =  australian,financial,review,reports,latest,news,business,finance,investment,politics,updated,real,time,reputation,independent,awardwinning,journalism,essential,reading,business,investor,community


category =  general
description =  axios,new,m